In [321]:
import boto3
import botocore
import os
from datetime import datetime

In [385]:
def make_dir(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
def getEssentials():
    keysdatafile = 'awskeys.txt'
    bucketname = 'uab-dropbox'

    # Go to ShareBox folder in user Documents Directory
    documentspath = os.path.join(os.path.expanduser('~'),"Documents\\")    
    boxpath = os.path.join(documentspath,"ShareBox\\")  
    # Create one if it does not exist
    make_dir(boxpath)
    
    return keysdatafile,bucketname,boxpath

def ReadCredentials(datafile):
    
    keyID=secretID=sessionToken=''
    
    with open(datafile) as f:
        i = 0
        for line in f:
            line = line.split("=") 
            if line:
                if i == 0: # aws_access_key_id
                    keyID = line[1].rstrip('\n')
                if i == 1: # aws_secret_access_key
                    secretID= line[1].rstrip('\n')      
                if i == 2: # aws_session_token
                    sessionToken = line[1].rstrip('\n')    
                i += 1

    return keyID,secretID,sessionToken

def listLocalDir(dirPath,boxPath,my_lst):
    entities = os.listdir(dirPath) # File or Folder
    for entity in entities:
        fullpath = os.path.abspath(os.path.join(dirPath,entity))
        isFile = os.path.isfile(fullpath)
        if isFile == False:
            listDir(fullpath,boxpath,my_lst)
        else:
            EntityPath = fullpath.replace(boxpath,'').replace(entity,'')
            if len(EntityPath) !=0:
                if EntityPath[-1] =='\\':
                    EntityPath = EntityPath[:-1]
            EntityPath = EntityPath.replace('\\','/')       
            my_dict = {'File':entity,'SubDir':EntityPath}
            my_lst.append(my_dict)
    return my_lst

def listFiles(keysdatafile,bucketname):
    documentspath = os.path.join(os.path.expanduser('~'),"Documents\\")    
    boxpath = os.path.join(documentspath,"ShareBox\\")  
    filelist =[]
    
    keyID,secretID,sessionToken = ReadCredentials(keysdatafile)
    resource = boto3.resource('s3',
                              aws_access_key_id = keyID,
                              aws_secret_access_key = secretID)

    bucket_of_interest = resource.Bucket(bucketname)
   
    for item in bucket_of_interest.objects.all():
        filelist.append(parseKey(item.key,item.last_modified))
        #print((datetime.strptime(str(item.last_modified), '%Y-%m-%d %H:%M:%S+00:00')))
    return filelist   

def parseKey(keyitem,lastmodified):
    tmp = keyitem.split('/')
    file = tmp[len(tmp) - 1]
    filename = file.split('\\')[len(file.split('\\'))-1]
    subdir = '/'.join(tmp)
    subdir = subdir.replace('/' + file,'')
    file_subDir_dict = {'File':file,'Filename':filename,'SubDir':subdir,'Key':keyitem,'Lastmodified':lastmodified}
    return file_subDir_dict

def deleteFileinS3(client,filebucket,file,dirname):
    delete_file_bucket = str(filebucket)
    delete_file_key = str(dirname) + '/' + str(file)
    response = client.delete_object(Bucket = delete_file_bucket,Key = delete_file_key)
    return response

def deleteAllFiles(keysdatafile,bucketname,delete_lst):
    keyID,secretID,sessionToken = ReadCredentials(keysdatafile)
    client = boto3.client('s3', aws_access_key_id = keyID, aws_secret_access_key = secretID)
    for item in delete_lst:
        #print('File:',item['File'],'SubDir:',item['SubDir'])
        response = deleteFileinS3(client,bucketname,item['File'],item['SubDir'])
        
def uploadtoS3(client,filebucket,file,dirname):
    try:
        upload_file_bucket = filebucket
        upload_file_key = str(dirname) + '/' + str(file)
        client.upload_file(file,upload_file_bucket,upload_file_key)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
            
def uploadAllFiles(keysdatafile,bucketname,upload_lst):
    keyID,secretID,sessionToken = ReadCredentials(keysdatafile)
    client = boto3.client('s3', aws_access_key_id = keyID, aws_secret_access_key = secretID)
    for item in upload_lst:
        filepath = os.path.abspath(os.path.join(boxpath,item['SubDir']))
        file = os.path.abspath(os.path.join(filepath,item['File']))
        uploadtoS3(client,bucketname,file,item['SubDir'])
        
        
def downloadAllFiles(keysdatafile,bucketname,download_lst,boxpath):
    #boxpath = 'C:\\Users\\sumanth.bail\\Documents\\Test\\' # ---> Test for downloading to different location i.e different machine
    keyID,secretID,sessionToken = ReadCredentials(keysdatafile)
    client = boto3.client('s3', aws_access_key_id = keyID, aws_secret_access_key = secretID)
    for item in download_lst:
        download_file_key = item['Key'] 
        #download_file_name = item['File']
        download_file_name = os.path.abspath(os.path.join(boxpath,item['SubDir']))
        make_dir(download_file_name)
        download_file_name = os.path.abspath(os.path.join(download_file_name,item['Filename']))
        #print(download_file_name)
        client.download_file(bucketname,download_file_key,download_file_name)

def downloadModifiedFilesSinceLastRun(keysdatafile,bucketname,download_lst,boxpath,LastRun_utc):
    keyID,secretID,sessionToken = ReadCredentials(keysdatafile)
    client = boto3.client('s3', aws_access_key_id = keyID, aws_secret_access_key = secretID)
    for item in download_lst:
        download_file_key = item['Key'] 
        download_file_name = os.path.abspath(os.path.join(boxpath,item['SubDir']))
        make_dir(download_file_name)
        download_file_name = os.path.abspath(os.path.join(download_file_name,item['Filename']))
        if (datetime.strptime(str(item['Lastmodified']), '%Y-%m-%d %H:%M:%S+00:00') > LastRun_utc) or os.path.exists(download_file_name) == False:
            print('Successful File Download: ',download_file_name)
            client.download_file(bucketname,download_file_key,download_file_name)
        else:
            print(download_file_name,' No file change in S3. File exists locally. Download skipped!')
       

#### List Files in S3 (With Delete ALL if needed)

In [372]:
keysdatafile,bucketname,boxpath = getEssentials()
s3_file_dir_lst = listFiles(keysdatafile,bucketname)

print('# of items in S3: ', len(s3_file_dir_lst), 'Last Run',LastRun_utc)

isDelete = False
if isDelete == True:
    deleteAllFiles(keysdatafile,bucketname,s3_file_dir_lst)
else:
    print('Delete switch turned off!')

#print(s3_file_dir_lst)
   

# of items in S3:  11 Last Run 2020-07-05 15:23:31.829955
Delete switch turned off!


#### List local Files and Upload to S3

In [360]:
keysdatafile,bucketname,boxpath = getEssentials()

# Get list of Files and the folders they reside under from ShareBox
local_lst = []
local_lst = listLocalDir(boxpath,boxpath,local_lst)

isUpload = True
if isUpload == True:
    uploadAllFiles(keysdatafile,bucketname,local_lst)
else:
    print('Upload switch turned off!')

#### Get List S3 Files and Download to local Sharebox (Calls for: Only those modified since last run or All)

In [384]:
keysdatafile,bucketname,boxpath = getEssentials()

s3_file_dir_lst = listFiles(keysdatafile,bucketname)

#print(s3_file_dir_lst)
isDownload = True
if isDownload == True:
    downloadModifiedFilesSinceLastRun(keysdatafile,bucketname,s3_file_dir_lst,boxpath,LastRun_utc)
    LastRun_utc = datetime.utcnow()
else:
    print('Download flag is turned off!')

C:\Users\sumanth.bail\Documents\ShareBox\Nationwide.txt  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\Statewide.txt  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\Zipcode.txt  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\LocPictures\Pictures\Archive\dog.jpg  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\LocPictures\Pictures\Archive\icon96.png  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\LocPictures\Pictures\Archive\indoor.jpg  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\LocPictures\Pictures\building.jpg  No file change in S3. File exists locally. Download skipped!
C:\Users\sumanth.bail\Documents\ShareBox\LocPictures\Pictures\coast.jpg  No file change